# Aprendizagem supervisionada: Regressão

Tal como a classificação, a regressão é uma das abordagens usadas no contexto da aprendizagem supervisionada. Neste caso, o foco é na previsão de valores contínuos em vez de classes ou categorias discretas. Este tutorial foca-se na aplicação e avaliação de diferentes algoritmos de regressão disponibilizados pela biblioteca [scikit-learn](https://scikit-learn.org/).

In [ ]:
import sklearn

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme()

## Preparação dos dados

Por uma questão de simplicidade, neste tutorial vamos usar a função `make_regression` para gerar um conjunto de dados sintético com apenas uma característica e o correspondente valor alvo:

In [ ]:
from sklearn.datasets import make_regression

In [ ]:
X, y = make_regression(1000, 1, noise=10, random_state=37)
df = pd.DataFrame(X, columns=['x'])
df['y'] = y

df.head()

Para estimar o desempenho de diferentes modelos de regressão e avaliar a sua capacidade de generalização para dados que não foram vistos, vamos particionar este conjunto de forma a que 80% dos dados sejam usados para treino e os restantes 20% para teste:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

Podemos visualizar os dois conjuntos para verificar se têm uma distribuição semelhante:

In [ ]:
sns.scatterplot(df_train, x='x', y='y')
plt.title('Training Set')
plt.show()

sns.scatterplot(df_test, x='x', y='y')
plt.title('Test Set')
plt.show()

## Treino

O algoritmo de regressão mais usado é a regressão linear. Na biblioteca *scikit-learn* esta é implementada pela classe [`Linear Regression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Para além disso, a maioria dos algoritmos que exploramos para classificação têm uma versão que pode ser usada para problemas de regressão. Por exemplo:

- k-NN: [`KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
- Árvores de Decisão: [`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)
- Random Forest: [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.htm)
- Máquinas de Vetores de Suporte: [`SVR`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)
- Redes Neuronais: [`MLPRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)

Vamos explorar o uso destes algoritmos para treinar modelos de regressão no conjunto de dados definido anteriormente.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [ ]:
regressors = {
    'Linear Regression': LinearRegression(),
    'k-NN': KNeighborsRegressor(n_neighbors=3),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=10),
    'SVM': SVR(kernel='linear'),
    'MLP': MLPRegressor(hidden_layer_sizes=(8,), activation='tanh', max_iter=20000)
}

**Nota**: Os hiperparametros que podem ser usados para configurar o treino de modelos usando cada um destes algoritmos são semelhantes aos dos seus equivalentes para classificação. Acima, na listagem dos algoritmos e das respectivas classes que os implementam, é possível aceder à documentação da classe clicando no seu nome. Tal como para a classificação, recomenda-se a exploração de diferentes configurações dos algoritmos. 

**Nota**: A biblioteca *scikit-learn* também implementa versões da regressão linear com regularização: [`Ridge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html), [`Lasso`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) e [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).

Vamos agora aplicar uma estratégia de validação cruzada para analisar o desempenho de cada uma das abordagens. No contexto da regressão, não existe uma função equivalente à `classification_report` usada para classificação. Como tal, por simplicidade, vamos usar a média dos valores devolvidos pela função `cross_val_score`. Neste caso, os valores correspondem ao coeficiente de determinação (R<sup>2</sup>). Outras métricas serão exploradas mais à frente, quando os modelos forem avaliados no conjunto de [teste](#Teste).

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
for name, regressor in regressors.items():
    cv_scores = cross_val_score(regressor, df_train[['x']], df_train['y'], cv=5)
    print(f'{name}: {np.mean(cv_scores)}')

Após encontrarmos a melhor configuração (ou uma configuração satisfatória) para cada algoritmo, podemos então treinar os modelos no conjunto de treino completo usando essas configurações:

In [ ]:
for regressor in regressors.values():
    regressor.fit(df_train[['x']], df_train['y'])

Como o nosso conjunto de dados tem apenas uma característica, podemos visualizar diretamente a curva da regressão aprendida por cada modelo:

In [ ]:
def draw_regression(df, regressor, title):
    draw_df = pd.DataFrame()
    draw_df['x'] = np.arange(min(df['x'])-0.5, max(df['x'])+0.5, 0.01)
    draw_df['y'] = regressor.predict(draw_df[['x']])
    sns.scatterplot(df, x='x', y='y')
    sns.lineplot(draw_df, x='x', y='y', color='orange')
    plt.title(title)
    plt.show()

In [ ]:
for name, regressor in regressors.items():
    draw_regression(df_train, regressor, name)

## Teste

Tal como para a classificação, após explorar várias abordagens no conjunto de treino, podemos selecionar as melhores e avaliar o seu desempenho no conjunto de teste. Neste caso, vamos usar todos os modelos treinados anteriormente. Para além do coeficiente de determinação (R<sup>2</sup>), vamos também analisar o desempenho usando o erro quadrático médio (MSE) e o erro absoluto médio (MAE). O coeficiente de determinação mede a proporção da variabilidade da variável dependente que pode ser explicada pelas variáveis independentes e, portanto, deve ser maximizado. Por outro lado, as duas outras métricas medem o erro médio da previsão e, portanto, devem ser minimizadas. A principal diferença entre as duas é que o erro quadrático médio dá mais peso a disparidades maiores entre o valor previsto e o esperado. Analisando todas estas métricas em conjunto podemos obter uma visão mais completa e detalhada do desempenho dos modelos de regressão e tomar decisões mais informadas sobre sua adequação para o problema em questão.

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
test_metrics = {}
for name, regressor in regressors.items():
    y_pred = regressor.predict(df_test[['x']])
    test_metrics[name] = {
        'r2': r2_score(df_test['y'], y_pred),
        'mse': mean_squared_error(df_test['y'], y_pred),
        'mae': mean_absolute_error(df_test['y'], y_pred)
    }

eval_df = pd.DataFrame.from_dict(test_metrics, orient='index')
eval_df